# Update attributes on reviewed parcels

In [1]:
import arcpy
from datetime import datetime
import os
import pandas as pd
from arcgis import GIS
import numpy as np
from arcgis.features import GeoAccessor, GeoSeriesAccessor
arcpy.env.overwriteOutput = True

# show all columns
pd.options.display.max_columns = None

# pd.DataFrame.spatial.from_featureclass(???)
# df.spatial.to_featureclass(location=???,sanitize_columns=False)

In [2]:
# path output folder
outputs = '.\\Outputs'

In [3]:
# create output gdb
outputs = '.\\Outputs'
gdb = os.path.join(outputs, "wfrc_hui_20211101.gdb")
if not arcpy.Exists(gdb):
    arcpy.CreateFileGDB_management(outputs, "wfrc_hui_20211101.gdb")
    
scratch = os.path.join(outputs, "scratch_rp.gdb")
if not arcpy.Exists(scratch):
    arcpy.CreateFileGDB_management(outputs, "scratch_rp.gdb")

In [4]:
# load data
hui = r"E:\Projects\Housing-Unit-Inventory\Review\reviewed.gdb\hui_reviewed_and_nonreviewed_parcels"
hui_sdf = pd.DataFrame.spatial.from_featureclass(hui)
hui_sdf.shape

(391825, 26)

In [5]:
hui_sdf.columns

Index(['OBJECTID', 'PARCEL_ID', 'TYPE_WFRC', 'SUBTYPE_WFRC', 'NOTE', 'IS_OUG',
       'CITY', 'SUBREGION', 'COUNTY', 'HOUSE_CNT', 'ADDR_CNT', 'UNIT_COUNT',
       'PARCEL_COUNT', 'FLAG', 'DUA', 'FLOORS_CNT', 'PARCEL_ACRES',
       'BLDG_SQFT', 'TOTAL_MKT_VALUE', 'BUILT_YR', 'BUILT_DECADE', 'UNIT_ID',
       'ADJUSTED', 'REVIEWED', 'MERGE_SRC', 'SHAPE'],
      dtype='object')

In [6]:
# regenerate unique building id
hui_sdf['UNIT_ID'] = hui_sdf.index + 1

In [7]:
# update the floor counts
hui_sdf.loc[(hui_sdf['FLOORS_CNT'] < 1) | (hui_sdf['FLOORS_CNT'].isnull() == True), 'FLOORS_CNT'] = 1
hui_sdf['FLOORS_CNT'] = hui_sdf['FLOORS_CNT'].round()

In [8]:
# remove parcels with zero housing units and store them for later
vacant = hui_sdf[hui_sdf['UNIT_COUNT'] == 0].copy()
vacant.spatial.to_featureclass(location=os.path.join(scratch, 'vacant'),sanitize_columns=False)
hui_sdf = hui_sdf[~(hui_sdf['UNIT_COUNT'] == 0)].copy()

print(vacant.shape)
print(hui_sdf.shape)

(95, 26)
(391730, 26)


In [9]:
# remove parcels from other categories
office = hui_sdf[hui_sdf['TYPE_WFRC'] == 'office'].copy()
group_quarters = hui_sdf[hui_sdf['TYPE_WFRC'] == 'group_quarters'].copy()
retail = hui_sdf[hui_sdf['TYPE_WFRC'] == 'retail'].copy()

office.spatial.to_featureclass(location=os.path.join(scratch, 'office'),sanitize_columns=False)
group_quarters.spatial.to_featureclass(location=os.path.join(scratch, 'group_quarters'),sanitize_columns=False)
retail.spatial.to_featureclass(location=os.path.join(scratch, 'retail'),sanitize_columns=False)

hui_sdf = hui_sdf[~(hui_sdf['TYPE_WFRC'] == 'office')].copy()
hui_sdf = hui_sdf[~(hui_sdf['TYPE_WFRC'] == 'group_quarters')].copy()
hui_sdf = hui_sdf[~(hui_sdf['TYPE_WFRC'] == 'retail')].copy()


In [10]:
# parcels that had duplicate addr pts
parcels_duplicate_addr_pts = hui_sdf[(hui_sdf['ADJUSTED'].str.contains('Dupli') == True)|
                                     (hui_sdf['ADJUSTED'].str.contains('dupli') == True)].copy()

parcels_duplicate_addr_pts.spatial.to_featureclass(location=os.path.join(scratch, 'parcels_duplicate_addr_pts'),sanitize_columns=False)
parcels_duplicate_addr_pts.shape

(42, 26)

In [11]:
# set total market value to zero
hui_sdf['TOTAL_MKT_VALUE'] = np.nan
hui_sdf['BLDG_SQFT'] = np.nan

hui_sdf.spatial.to_featureclass(location=os.path.join(scratch, 'wfrc_housing_unit_inventory_20211101_draft'),
                                sanitize_columns=False)

'E:\\Projects\\Housing-Unit-Inventory\\Outputs\\scratch_rp.gdb\\wfrc_housing_unit_inventory_20211101_draft'

In [12]:
# update attributes
hui = os.path.join(scratch, 'wfrc_housing_unit_inventory_20211101_draft')

# perform spatial join with within query and sum total market value
weber_parcels = r'.\2020-Weber\Inputs\Utah_weber_County_Parcels_LIR.gdb\Parcels_Weber_LIR_UTM12'
davis_parcels = r'.\2020-Davis\Inputs\Davis_County_LIR_Parcels.gdb\Parcels_Davis_LIR_UTM12'
sl_parcels = r'.\2020-SaltLake\Inputs\Utah_Salt_Lake_County_Parcels_LIR.gdb\Parcels_SaltLake_LIR_UTM12'

# merge all parcels for sj
merged_parcels = arcpy.Merge_management([weber_parcels, davis_parcels, sl_parcels], os.path.join(scratch, 'merged_parcels'))

merged_parcels_pts = arcpy.FeatureToPoint_management(merged_parcels, os.path.join(scratch, 'merged_parcels_pts'), "INSIDE")

In [13]:
# use spatial join to recalc market value and bldg sqft
target_features = hui
join_features = merged_parcels_pts
output_features = os.path.join(scratch, "hui_parcels_sj")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# total market value
fieldindex = fieldmappings.findFieldMapIndex('TOTAL_MKT_VALUE')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# building square ft
fieldindex = fieldmappings.findFieldMapIndex('BLDG_SQFT')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# run the spatial join
sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_COMMON", 
                           fieldmappings, "INTERSECT")

In [14]:
# load sj as sdf and format
sj_sdf = pd.DataFrame.spatial.from_featureclass(sj)
sj_sdf = sj_sdf[['UNIT_ID','TOTAL_MKT_VALUE','BLDG_SQFT']].copy()
sj_sdf.columns = ['UNIT_ID','TOTAL_MKT_VALUE_NEW','BLDG_SQFT_NEW']
sj_sdf.shape

(391713, 3)

In [15]:
# add geographies back
cities = r'.\Geographies\Cities.shp'
counties = r'.\Geographies\Counties.shp'
subcounties = r'.\Geographies\SubCountyArea_2019.shp'

geographies = [(counties,'COUNTY'),(subcounties, 'SUBCOUNTY'),(cities, 'CITY')]

for geog in geographies:
    
    # use spatial join to summarize res units
    target_features = hui
    join_features = geog[0]
    output_features = os.path.join(scratch, "hui_{}".format(geog[1]))

    fieldmappings = arcpy.FieldMappings()
    fieldmappings.addTable(target_features)
    fieldmappings.addTable(join_features)

    # run the spatial join
    sj2 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                               fieldmappings, "WITHIN")
    
    sj2_sdf = pd.DataFrame.spatial.from_featureclass(sj2)
    sj2_sdf = sj2_sdf[['UNIT_ID','NAME']].copy()
    sj2_sdf.columns = ['UNIT_ID', geog[1]]
    hui_sdf = hui_sdf.merge(sj2_sdf, left_on = 'UNIT_ID', right_on = 'UNIT_ID', how='left')

In [16]:
# bring back tmv and bldg sqft attributes
new_hui = hui_sdf.merge(sj_sdf, left_on = 'UNIT_ID', right_on = 'UNIT_ID', how='left')
new_hui['TOTAL_MKT_VALUE'] = new_hui['TOTAL_MKT_VALUE_NEW']
new_hui['BLDG_SQFT'] = new_hui['BLDG_SQFT_NEW']

del new_hui['TOTAL_MKT_VALUE_NEW']
del new_hui['BLDG_SQFT_NEW']

In [17]:
# recalc built decade
new_hui.loc[(new_hui['BUILT_YR'] >= 1840) & (new_hui['BUILT_YR'] < 1850), 'BUILT_DECADE'] = "1840's"
new_hui.loc[(new_hui['BUILT_YR'] >= 1850) & (new_hui['BUILT_YR'] < 1860), 'BUILT_DECADE'] = "1850's"
new_hui.loc[(new_hui['BUILT_YR'] >= 1860) & (new_hui['BUILT_YR'] < 1870), 'BUILT_DECADE'] = "1860's"
new_hui.loc[(new_hui['BUILT_YR'] >= 1870) & (new_hui['BUILT_YR'] < 1880), 'BUILT_DECADE'] = "1870's"
new_hui.loc[(new_hui['BUILT_YR'] >= 1880) & (new_hui['BUILT_YR'] < 1890), 'BUILT_DECADE'] = "1880's"
new_hui.loc[(new_hui['BUILT_YR'] >= 1890) & (new_hui['BUILT_YR'] < 1900), 'BUILT_DECADE'] = "1890's"
new_hui.loc[(new_hui['BUILT_YR'] >= 1900) & (new_hui['BUILT_YR'] < 1910), 'BUILT_DECADE'] = "1900's"
new_hui.loc[(new_hui['BUILT_YR'] >= 1910) & (new_hui['BUILT_YR'] < 1920), 'BUILT_DECADE'] = "1910's"
new_hui.loc[(new_hui['BUILT_YR'] >= 1920) & (new_hui['BUILT_YR'] < 1930), 'BUILT_DECADE'] = "1920's"
new_hui.loc[(new_hui['BUILT_YR'] >= 1930) & (new_hui['BUILT_YR'] < 1940), 'BUILT_DECADE'] = "1930's"
new_hui.loc[(new_hui['BUILT_YR'] >= 1940) & (new_hui['BUILT_YR'] < 1950), 'BUILT_DECADE'] = "1940's"
new_hui.loc[(new_hui['BUILT_YR'] >= 1950) & (new_hui['BUILT_YR'] < 1960), 'BUILT_DECADE'] = "1950's"
new_hui.loc[(new_hui['BUILT_YR'] >= 1960) & (new_hui['BUILT_YR'] < 1970), 'BUILT_DECADE'] = "1960's"
new_hui.loc[(new_hui['BUILT_YR'] >= 1970) & (new_hui['BUILT_YR'] < 1980), 'BUILT_DECADE'] = "1970's"
new_hui.loc[(new_hui['BUILT_YR'] >= 1980) & (new_hui['BUILT_YR'] < 1990), 'BUILT_DECADE'] = "1980's"
new_hui.loc[(new_hui['BUILT_YR'] >= 1990) & (new_hui['BUILT_YR'] < 2000), 'BUILT_DECADE'] = "1990's"
new_hui.loc[(new_hui['BUILT_YR'] >= 2000) & (new_hui['BUILT_YR'] < 2010), 'BUILT_DECADE'] = "2000's"
new_hui.loc[(new_hui['BUILT_YR'] >= 2010) & (new_hui['BUILT_YR'] < 2020), 'BUILT_DECADE'] = "2010's"
new_hui.loc[(new_hui['BUILT_YR'] >= 2020) & (new_hui['BUILT_YR'] < 2030), 'BUILT_DECADE'] = "2020's"

In [18]:
new_hui = new_hui[[ 'PARCEL_ID', 'TYPE_WFRC', 'SUBTYPE_WFRC', 'NOTE', 'IS_OUG',
       'CITY_x', 'COUNTY_x', 'SUBCOUNTY', 'HOUSE_CNT', 'ADDR_CNT',
       'UNIT_COUNT', 'PARCEL_COUNT', 'DUA', 'FLOORS_CNT',
       'PARCEL_ACRES', 'BLDG_SQFT', 'TOTAL_MKT_VALUE', 'BUILT_YR',
       'BUILT_DECADE', 'UNIT_ID', 'SHAPE']].copy()

new_hui = new_hui.rename(columns={'CITY_x': 'CITY'})
new_hui = new_hui.rename(columns={'COUNTY_x': 'COUNTY'})

In [19]:
# recalc max dua
new_hui['DUA'] = (new_hui['UNIT_COUNT'] / new_hui['PARCEL_ACRES']).round(3)

In [20]:
# export to shape
new_hui.spatial.to_featureclass(location=os.path.join(gdb, 'wfrc_housing_unit_inventory_20211101'),sanitize_columns=False)

'E:\\Projects\\Housing-Unit-Inventory\\Outputs\\wfrc_hui_20211101.gdb\\wfrc_housing_unit_inventory_20211101'

In [21]:
# # read in taz households
# taz = r".\Outputs\wfrc_hui_20211025.gdb\taz_projections_hui_sj_wfrc"
# taz_sdf = pd.DataFrame.spatial.from_featureclass(taz)
# taz_sdf.columns

In [22]:
# taz_sdf = taz_sdf[['CityArea', 'ModelArea', 'CO_TAZID', 'DEVACRES', 'CO_FIPS', 'CO_NAME', 'TAZID', 'RELEASE',
#                    'YEAR2019','UNIT_COUNT', 'HUI', 'SHAPE']].copy()

# taz_sdf = taz_sdf.rename(columns={"YEAR2019": "REMM_2019"})
# taz_sdf = taz_sdf.rename(columns={"UNIT_COUNT": "HUI_2019"})


# # positive indicates REMM overpredicted, negative REMM indicates underpredicted
# taz_sdf['HH19_DIFF'] = taz_sdf['REMM_2019'] - taz_sdf['HUI_2019']
# taz_sdf['HH19_PCT_DIFF'] = taz_sdf['HH19_DIFF'] / taz_sdf['HUI_2019'] * 100


# taz_sdf.spatial.to_featureclass(location=os.path.join(gdb, 'remm_vs_hui_taz'),sanitize_columns=False)

In [23]:
# # summarize by city - positive indicates REMM overpredicted, negative REMM indicates underpredicted
# city_summary = taz_sdf.groupby('CityArea')[["REMM_2019",'HUI_2019']].sum().reset_index()
# city_summary['HH19_DIFF'] = city_summary['REMM_2019'] - city_summary['HUI_2019']
# city_summary['HH19_PCT_DIFF'] = city_summary['HH19_DIFF'] / city_summary['HUI_2019'] *100
# city_summary.to_csv(os.path.join(outputs, 'hh_city_summary.csv'))

In [24]:
# # summarize by county - positive indicates REMM overpredicted, negative REMM indicates underpredicted
# county_summary = taz_sdf.groupby('CO_NAME')[["REMM_2019",'HUI_2019']].sum().reset_index()
# county_summary['HH19_DIFF'] = county_summary['REMM_2019'] - county_summary['HUI_2019']
# county_summary['HH19_PCT_DIFF'] = county_summary['HH19_DIFF'] / county_summary['HUI_2019'] * 100
# county_summary.to_csv(os.path.join(outputs, 'hh_county_summary.csv'))